In [1]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

import torch
import torchtext
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

import pandas as pd
import re
from collections import Counter
from tqdm.notebook import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  # 能用gpu则用gpu

# 1.准备数据

In [2]:
class MyDataset(Dataset):
    def __init__(self, file_path, tokenizer, stopwords, debug=True):
        df = pd.read_csv(file_path)
        df = df.dropna().reset_index(drop=True)
        if debug:
            df = df.sample(2000).reset_index(drop=True)
        counter = Counter()
        sentences = []
        for title in tqdm(df['title']):
            # 去除标点符号
            title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
            tokens = [token for token in tokenizer(title.strip()) if token not in stopwords]
            counter.update(tokens)
            sentences.append(tokens)
        self.vocab = torchtext.vocab.vocab(counter, specials=['<unk>', '<pad>'])
        self.vocab.set_default_index(self.vocab['<unk>'])

        self.inputs = [self.vocab.lookup_indices(tokens) for tokens in sentences]
        self.labels = [[label] for label in df['label'].values.tolist()]
        self.n_class = len(df['label'].unique())

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return torch.LongTensor(self.inputs[idx]), torch.LongTensor(self.labels[idx])

In [3]:
file_path = '../data/THUCNews/train.csv'
tokenizer = torchtext.data.utils.get_tokenizer('spacy', language='zh_core_web_sm')
stopwords = [line.strip() for line in open('../stopwords/cn_stopwords.txt', 'r', encoding='utf-8').readlines()]
dataset = MyDataset(file_path, tokenizer, stopwords, debug=True)


def collate_fn(batch_data):
    return pad_sequence([x for x, y in batch_data], padding_value=1), torch.tensor([y for x, y in batch_data]).unsqueeze(1)


dataloader = DataLoader(dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)

  0%|          | 0/2000 [00:00<?, ?it/s]

# 2.构建模型

In [4]:
class TextCNN(nn.Module):
    def __init__(self, vocab_size, embed_size, n_class):
        super().__init__()
        # 嵌入层
        self.embedding = nn.Embedding(vocab_size, embed_size)
        # 输入通道为1，卷成16通道输出，卷积核大小为(3*embed_size)，3类似于n-gram，可以换
        self.conv = nn.Conv2d(1, 16, (3, embed_size))
        self.dropout = nn.Dropout(0.2)
        # 输出头
        self.fc = nn.Linear(16, n_class)

    def forward(self, x):  # x: [batch_size * 句子长度]
        x = x.permute(1, 0)
        x = self.embedding(x)  # [batch_size * 句子长度 * embed_size]
        x = x.unsqueeze(1)  # [batch_size * 1 * 句子长度 * embed_size]，加一个维度，用于卷积层的输入
        x = self.conv(x)  # [batch_size * 16(卷积层输出通道数) * 8(卷积后的宽) * 1(卷积后的高)]
        x = x.squeeze(3)  # [batch_size * 16(卷积层输出通道数) * 8(卷积后的宽)] 压缩大小为1的维度
        x = torch.relu(x)  # 激活函数，尺寸不变
        x = torch.max_pool1d(x, x.size(2))  # 在每个通道做最大池化，[batch_size * 16(卷积层输出通道数) * 1]
        x = x.squeeze(2)  # 压缩维度2，[batch_size * 16(卷积层输出通道数)]
        x = self.dropout(x)  # dropout，尺寸不变
        logits = self.fc(x)  # 全连接输出头，[batch_size * n_class]
        return logits


model = TextCNN(vocab_size=len(dataset.vocab), embed_size=512, n_class=dataset.n_class).to(device)
print(model)

TextCNN(
  (embedding): Embedding(8395, 512)
  (conv): Conv2d(1, 16, kernel_size=(3, 512), stride=(1, 1))
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=16, out_features=14, bias=True)
)


# 3.训练模型

In [5]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
model.train()
for epoch in range(5):
    for feature, target in dataloader:
        feature = feature.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        logits = model(feature)
        loss = criterion(logits, target.squeeze())
        loss.backward()
        optimizer.step()
    print('epoch:', epoch + 1, ', loss:', loss.item())

epoch: 1 , loss: 1.6783087253570557
epoch: 2 , loss: 1.125728964805603
epoch: 3 , loss: 1.435263752937317
epoch: 4 , loss: 1.0876044034957886
epoch: 5 , loss: 0.2656219005584717


# 4.预测

In [6]:
model.eval()
df_train = pd.read_csv('../data/THUCNews/train.csv')
df_test = pd.read_csv('../data/THUCNews/test.csv')

In [7]:
for i, row in df_test.sample(10).iterrows():
    title = row['title']
    actual = row['class']
    title = re.sub(r'[^\u4e00-\u9fa5]', '', title)
    tokens = [token for token in tokenizer(title.strip()) if token not in stopwords]
    inputs = dataset.vocab.lookup_indices(tokens)
    inputs = torch.LongTensor(inputs).unsqueeze(1)
    predict = model(inputs)
    predict_class = dict(zip(df_train['label'], df_train['class']))[predict.max(1)[1].item()]    
    print(' '.join(tokens), '||| actual:', actual, ', predict:', predict_class)

[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


内置 重力 加速 详解 原道 智能 六大 优势 ||| actual: 科技 , predict: 体育
高校 干事 挪百万 公款 年 问杀 知情者 灭口 ||| actual: 社会 , predict: 娱乐
支持 倍光学 变焦 奥林巴斯 仅 售元 ||| actual: 科技 , predict: 科技
国足 冷落 西行誓 做 疯狂 牛曹 阳领队 象棋 大师 ||| actual: 体育 , predict: 体育
巴基斯坦 塔利班 新 头目 可能 丧生 ||| actual: 时政 , predict: 时政
随地 管理 店铺 淘宝 推出 款定 制手机 ||| actual: 科技 , predict: 科技
电力 商季 利润 超 预期 增长 ||| actual: 股票 , predict: 科技
仙剑 语音包 最新 宣传 视频 曝光 ||| actual: 游戏 , predict: 娱乐
组图 穿上 超萌 熊猫 元素 服饰 变身 潘达 控 ||| actual: 时尚 , predict: 娱乐
溢 指续 弹点 中海 发展 近月 狂泻 半数 市值 ||| actual: 股票 , predict: 科技
